In [ ]:
import time
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import numpy as np

In [5]:
# 生成标准数据

from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
df = pd.DataFrame()

alpha_name = "AlphaNet_1130"
for f, _, i in walk("/home/wuwenjun/Alpha_Factor"):
    for j in tqdm(i):
        df = pd.concat([df,pd.read_csv(f + "/" + j)],axis=0)
df.rename(columns={'AlphaNet': alpha_name}, inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['symbol'] =  df['symbol'].apply(lambda x : '{:0>6d}'.format(x))
df['symbol'] = df['symbol'].astype('string')
display(df)
# output
output_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun'
convert_to_standard_daily_feature_csv(alpha_name, df, output_path)

100%|██████████| 5/5 [00:00<00:00, 18.73it/s]


,symbol,timestamp,AlphaNet_1130
0,600872,2019-05-16 09:30:00,-0.010705
1,600872,2019-05-21 09:30:00,0.150820
2,600872,2019-05-24 09:30:00,-0.297267
3,600872,2019-05-29 09:30:00,0.203635
4,600872,2019-06-03 09:30:00,-0.043720
...,...,...,...
99067,600179,2021-06-24 09:30:00,-0.143311
99068,600243,2021-06-24 09:30:00,-0.088673
99069,002426,2021-06-24 09:30:00,-0.454667
99070,000536,2021-06-25 09:30:00,0.210734


In [6]:
import time
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
import numpy as np
 
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/wuwenjun/factor_lib")


alpha_list = list(FT.features_in_path.keys())
start_date = '2018-01-01'
end_date = '2021-03-02'
FT.load_feature_from_file(alpha_list, start_date, end_date, universe='Investable',
                                       timedelta=None, transformer=norm.standard_scale)
FT.load_return_data()

for alpha_name in alpha_list:
    FT.get_intersection_ic(feature_name=alpha_name, truncate_fold=None, method='spearman',
                                     period=('1d', '3d', '5d'))
    ic_flag, trading_direction = FT.test_ic(alpha_name, verbose=False)
    df, df_all = FT.get_ic_summary_by_month(num=6)
    if trading_direction == -1:
        negative = True
    else:
        negative = False
    FT.get_top_return(alpha_name, negative=negative, trade_type='long-short', transmission_rate=0.0,
                            tax_rate=0.0, verbose=True)

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 344/344 [00:00<00:00, 713.91it/s]



IC table for feature "AlphaNet_1130" (mean std IR):
+-------------------+------------------+------------------+------------------+------------------+------------------+
|      period       |  IC_AlphaNet_1d  |  IC_AlphaNet_2d  |  IC_AlphaNet_3d  |  IC_AlphaNet_4d  |  IC_AlphaNet_5d  |
+===================+==================+==================+==================+==================+==================+
| 20190516-20191031 |  3.94 5.88 0.67  |  4.99 5.35 0.93  |  5.37 5.29 1.02  |  5.51 5.18 1.06  |  5.21 5.29 0.98  |
+-------------------+------------------+------------------+------------------+------------------+------------------+
| 20191101-20200430 |  4.07 7.76 0.52  |  5.23 7.30 0.72  |  6.40 6.52 0.98  |  7.32 6.63 1.10  |  8.05 6.06 1.33  |
+-------------------+------------------+------------------+------------------+------------------+------------------+
| 20200501-20201031 |  2.88 6.11 0.47  |  4.24 6.42 0.66  |  4.60 5.88 0.78  |  4.96 5.53 0.90  |  5.62 5.28 1.06  |
+----------

Getting BT result for date: 20210301...: 100%|█████████████████████████████████████████████████████████████████████| 766/766 [00:07<00:00, 100.98it/s]



AlphaNet_1130 back-test result for long-short mode:

Back test summary: "long-short" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+--------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period       |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+==========================+===================+============+
| 20180102-20180630 |      0      |   0.00   |   0.00    |    0.00    |   nan    |   100.00    | 0.00(None    -None    )  |  0.00(None    )   |    0.00    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+--------------------------+-------------------+------------+
| 20180701-20181231 |      0      |   0.00   |   0.00    |    0.00    |   nan    |   100.00    | 0.00(None    -None    )  | 